In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import utils
from tensorflow.keras import mixed_precision
import os
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
mixed_precision.set_global_policy('mixed_float16')
import logging
tf.get_logger().setLevel(logging.ERROR)

2024-07-25 18:50:29.443669: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-25 18:50:29.468501: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 18:50:29.468529: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-25 18:50:29.468555: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-25 18:50:29.473893: I tensorflow/core/platform/cpu_feature_g

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070 Laptop GPU, compute capability 8.9


2024-07-25 18:50:31.119907: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-25 18:50:31.143959: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-25 18:50:31.147237: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
"""
#Batching using prefetch
train_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)
test_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)
"""

'\n#Batching using prefetch\ntrain_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)\ntest_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)\n'

In [3]:
fundus_train = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split1/train"
fundus_test = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split1/test"


BATCH_SIZE = 32
IMG_SIZE = (224, 224)


In [4]:
print(os.listdir(fundus_train))

['glaucoma', 'normal', 'cataract', 'diabetic_retinopathy']


In [5]:
IMG_HEIGHT, IMG_WIDTH = 224, 224

In [6]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset='training'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,
    validation_split=0.3,
    subset='validation'
)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_test,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,

)

Found 3372 files belonging to 4 classes.
Using 2361 files for training.


2024-07-25 18:50:35.782371: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-25 18:50:35.786517: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-25 18:50:35.790135: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Found 3372 files belonging to 4 classes.
Using 1011 files for validation.
Found 845 files belonging to 4 classes.


In [7]:
train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [8]:
#Using tf prefetch dataset
preprocess_input = tf.keras.applications.mobilenet_v3.preprocess_input

In [9]:
train_datagen = train_dataset.map(lambda x, y: (preprocess_input(x), y))
val_datagen = validation_dataset.map(lambda x, y: (preprocess_input(x), y))
test_datagen = test_dataset.map(lambda x, y: (preprocess_input(x), y))

In [10]:
train_datagen

<_MapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [11]:
import tensorflow.keras.applications as apps
IMG_WIDTH, IMG_HEIGHT = 224, 224
base_model = apps.MobileNetV3Large(weights = 'imagenet', include_top = False, input_shape = (IMG_WIDTH, IMG_HEIGHT, 3))
base_model.trainable = False

In [12]:

for i in base_model.layers:
    print(f'Layer: {i.name}, {i.trainable}')

Layer: input_1, False
Layer: rescaling, False
Layer: Conv, False
Layer: Conv/BatchNorm, False
Layer: tf.math.add, False
Layer: re_lu, False
Layer: tf.math.multiply, False
Layer: multiply, False
Layer: expanded_conv/depthwise, False
Layer: expanded_conv/depthwise/BatchNorm, False
Layer: re_lu_1, False
Layer: expanded_conv/project, False
Layer: expanded_conv/project/BatchNorm, False
Layer: expanded_conv/Add, False
Layer: expanded_conv_1/expand, False
Layer: expanded_conv_1/expand/BatchNorm, False
Layer: re_lu_2, False
Layer: expanded_conv_1/depthwise/pad, False
Layer: expanded_conv_1/depthwise, False
Layer: expanded_conv_1/depthwise/BatchNorm, False
Layer: re_lu_3, False
Layer: expanded_conv_1/project, False
Layer: expanded_conv_1/project/BatchNorm, False
Layer: expanded_conv_2/expand, False
Layer: expanded_conv_2/expand/BatchNorm, False
Layer: re_lu_4, False
Layer: expanded_conv_2/depthwise, False
Layer: expanded_conv_2/depthwise/BatchNorm, False
Layer: re_lu_5, False
Layer: expanded_co

In [13]:
len(base_model.layers)

263

In [14]:
No_of_classes = len(os.listdir(fundus_train))
No_of_classes

4

In [15]:
aug_layer = utils.return_data_aug_layer_for_eff_net()

In [16]:
from tensorflow.keras.layers import Dense, GlobalAvgPool2D, Input, Dropout
from tensorflow.keras.models import Model
inputs = Input(shape = (IMG_HEIGHT, IMG_WIDTH, 3), name = 'Input_layer')
x = aug_layer(inputs)
x = base_model(x, training = False)
x = layers.GlobalAvgPool2D()(x)


x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
Outputs = Dense(No_of_classes, activation = 'softmax', dtype = tf.float32)(x)

model_1 = Model(inputs, Outputs, name = 'MobilenetV3L_87.7')

model_1.summary()

Model: "MobilenetV3L"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 MobilenetV3large (Function  (None, 7, 7, 960)         2996352   
 al)                                                             
                                                                 
 global_average_pooling2d (  (None, 960)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               246016    
                                                      

In [17]:
#Model checkpointing
from tensorflow.keras.callbacks import ModelCheckpoint
model_1chkpt = ModelCheckpoint(filepath = os.path.join('Trained_Models','MobilenetV3L_87.7'), save_weights_only = False, save_best_only = True, verbose = 1)

In [18]:
from keras.src.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(factor=0.3, patience=2) 

In [19]:
#model compilation
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])

In [20]:
history_1 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = 10, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 1/10


2024-07-25 18:51:03.075647: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-07-25 18:51:03.149120: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-07-25 18:51:03.972906: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7da0800036e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-25 18:51:03.972944: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-07-25 18:51:03.979510: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-25 18:51:04.045269: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


74/74 [==============================] - ETA: 0s - loss: 1.1401 - accuracy: 0.4833
Epoch 1: val_loss improved from inf to 1.27264, saving model to Trained_Models/MobilenetV3L
74/74 [==============================] - 30s 355ms/step - loss: 1.1401 - accuracy: 0.4833 - val_loss: 1.2726 - val_accuracy: 0.3947 - lr: 0.0010
Epoch 2/10
74/74 [==============================] - ETA: 0s - loss: 0.8293 - accuracy: 0.6485
Epoch 2: val_loss improved from 1.27264 to 0.84173, saving model to Trained_Models/MobilenetV3L
74/74 [==============================] - 24s 322ms/step - loss: 0.8293 - accuracy: 0.6485 - val_loss: 0.8417 - val_accuracy: 0.7052 - lr: 0.0010
Epoch 3/10
74/74 [==============================] - ETA: 0s - loss: 0.6703 - accuracy: 0.7319
Epoch 3: val_loss did not improve from 0.84173
74/74 [==============================] - 15s 198ms/step - loss: 0.6703 - accuracy: 0.7319 - val_loss: 1.2022 - val_accuracy: 0.4332 - lr: 0.0010
Epoch 4/10
74/74 [==============================] - ETA: 0s

In [21]:
base_model.trainable = True

In [22]:
for layer in base_model.layers[:-50]:
    layer.trainable = False

In [23]:
for layer in base_model.layers:
    print(layer.trainable)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [24]:
start_epoch = 10
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.0003), metrics = ['accuracy'])
model_1.summary()

Model: "MobilenetV3L"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 MobilenetV3large (Function  (None, 7, 7, 960)         2996352   
 al)                                                             
                                                                 
 global_average_pooling2d (  (None, 960)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               246016    
                                                      

In [25]:
history_50 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+15, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 11/25
74/74 [==============================] - ETA: 0s - loss: 0.5216 - accuracy: 0.8115
Epoch 11: val_loss did not improve from 0.66410
74/74 [==============================] - 17s 163ms/step - loss: 0.5216 - accuracy: 0.8115 - val_loss: 0.7823 - val_accuracy: 0.6706 - lr: 3.0000e-04
Epoch 12/25
74/74 [==============================] - ETA: 0s - loss: 0.4415 - accuracy: 0.8348
Epoch 12: val_loss did not improve from 0.66410
74/74 [==============================] - 11s 150ms/step - loss: 0.4415 - accuracy: 0.8348 - val_loss: 0.8056 - val_accuracy: 0.6736 - lr: 3.0000e-04
Epoch 13/25
74/74 [==============================] - ETA: 0s - loss: 0.4236 - accuracy: 0.8399
Epoch 13: val_loss did not improve from 0.66410
74/74 [==============================] - 13s 163ms/step - loss: 0.4236 - accuracy: 0.8399 - val_loss: 1.0192 - val_accuracy: 0.4807 - lr: 3.0000e-04
Epoch 14/25
74/74 [==============================] - ETA: 0s - loss: 0.3411 - accuracy: 0.8767
Epoch 14: val_loss improved f

In [26]:
start_epoch = 25
for layer in base_model.layers[-125:]:
    layer.trainable = True

In [27]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.00003), metrics = ['accuracy'])
model_1.summary()

Model: "MobilenetV3L"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 MobilenetV3large (Function  (None, 7, 7, 960)         2996352   
 al)                                                             
                                                                 
 global_average_pooling2d (  (None, 960)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               246016    
                                                      

In [28]:
history_100 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+15, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 26/40
74/74 [==============================] - ETA: 0s - loss: 0.2685 - accuracy: 0.9068
Epoch 26: val_loss did not improve from 0.41639
74/74 [==============================] - 23s 186ms/step - loss: 0.2685 - accuracy: 0.9068 - val_loss: 0.7870 - val_accuracy: 0.6726 - lr: 3.0000e-05
Epoch 27/40
74/74 [==============================] - ETA: 0s - loss: 0.2541 - accuracy: 0.9123
Epoch 27: val_loss did not improve from 0.41639
74/74 [==============================] - 10s 127ms/step - loss: 0.2541 - accuracy: 0.9123 - val_loss: 0.4772 - val_accuracy: 0.8358 - lr: 3.0000e-05
Epoch 28/40
74/74 [==============================] - ETA: 0s - loss: 0.2478 - accuracy: 0.9047
Epoch 28: val_loss improved from 0.41639 to 0.32292, saving model to Trained_Models/MobilenetV3L
74/74 [==============================] - 17s 232ms/step - loss: 0.2478 - accuracy: 0.9047 - val_loss: 0.3229 - val_accuracy: 0.9050 - lr: 3.0000e-05
Epoch 29/40
74/74 [==============================] - ETA: 0s - loss: 0.2519

In [29]:
start_epoch = 40
for layer in base_model.layers[-250:]:
    layer.trainable = True

In [30]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.00009), metrics = ['accuracy'])
model_1.summary()

Model: "MobilenetV3L"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 MobilenetV3large (Function  (None, 7, 7, 960)         2996352   
 al)                                                             
                                                                 
 global_average_pooling2d (  (None, 960)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               246016    
                                                      

In [31]:
history_200 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+50, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 41/90
74/74 [==============================] - ETA: 0s - loss: 0.3224 - accuracy: 0.8937
Epoch 41: val_loss did not improve from 0.26771
74/74 [==============================] - 42s 421ms/step - loss: 0.3224 - accuracy: 0.8937 - val_loss: 0.4803 - val_accuracy: 0.8694 - lr: 9.0000e-05
Epoch 42/90
74/74 [==============================] - ETA: 0s - loss: 0.2550 - accuracy: 0.9119
Epoch 42: val_loss did not improve from 0.26771
74/74 [==============================] - 23s 306ms/step - loss: 0.2550 - accuracy: 0.9119 - val_loss: 0.7618 - val_accuracy: 0.7201 - lr: 9.0000e-05
Epoch 43/90
74/74 [==============================] - ETA: 0s - loss: 0.2707 - accuracy: 0.9111
Epoch 43: val_loss did not improve from 0.26771
74/74 [==============================] - 20s 272ms/step - loss: 0.2707 - accuracy: 0.9111 - val_loss: 0.3554 - val_accuracy: 0.8912 - lr: 9.0000e-05
Epoch 44/90
74/74 [==============================] - ETA: 0s - loss: 0.2477 - accuracy: 0.9123
Epoch 44: val_loss did not im

KeyboardInterrupt: 

In [32]:
MobileNet_Best = tf.keras.models.load_model("/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/ML_Models/Trained_Models/MobilenetV3L_87.7")


In [33]:
MobileNet_Best.evaluate(test_datagen)

27/27 [==============================] - 1s 25ms/step - loss: 0.3689 - accuracy: 0.8769


[0.36894628405570984, 0.8769230842590332]